In [3]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))

import data_loader
import data_references
from energy_channels import EnergyChannel
import datetime
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import plot_tools
import numpy as np
import pandas as pd
import psd_tool
from field_models import model
import data_cache
import scipy


import importlib
importlib.reload(plot_tools)
importlib.reload(data_loader)
importlib.reload(data_references)
importlib.reload(psd_tool)
importlib.reload(data_cache)

%matplotlib qt

In [4]:
start = datetime.datetime(year = 2017, month = 5, day = 1)
end = datetime.datetime(year = 2017, month = 6, day = 7, hour=23, minute=59, second=59)

In [5]:

data_A = data_loader.load_compressed_rept_data(satellite="A", start=start, end=end)
data_B = data_loader.load_compressed_rept_data(satellite="B", start=start, end=end)

fesa_A = data_A["FESA"]
L_A = data_A["L"]
mlt_A = data_A["MLT"]
epoch_A = data_A["EPOCH"]
energies = data_A["ENERGIES"]

fesa_B = data_B["FESA"]
L_B = data_B["L"]
mlt_B = data_B["MLT"]
epoch_B = data_B["EPOCH"]
energies = data_B["ENERGIES"]

Loading REPT data between: 2017-05-01 00:00:00 and 2017-06-07 23:59:59.
Loading : REPT_201705_A.npz
Loading : REPT_201706_A.npz
Loading REPT data between: 2017-05-01 00:00:00 and 2017-06-07 23:59:59.
Loading : REPT_201705_B.npz
Loading : REPT_201706_B.npz


In [6]:
l_cut = 3.5
tol = 0.25

satisfies_L_cut_A = ((l_cut - tol) < L_A) & (L_A < (l_cut + tol))
satisfies_L_cut_B = ((l_cut - tol) < L_B) & (L_B < (l_cut + tol))

fesa_A = fesa_A[:, (energies < 4.3)]
fesa_B = fesa_B[:, (energies < 4.3)]

fesa_A_DF = pd.DataFrame(fesa_A[satisfies_L_cut_A, :], index=epoch_A[satisfies_L_cut_A])
fesa_B_DF = pd.DataFrame(fesa_B[satisfies_L_cut_B, :], index=epoch_B[satisfies_L_cut_B])

In [7]:
fesa = pd.concat([fesa_A_DF, fesa_B_DF], sort=False).sort_index()
fesa = fesa.resample("5h").mean()
#FESA = FESA.rolling(window=avg_t, center=True).mean()

In [8]:
omni_1_min = data_loader.load_omni_data_1min_res(start=start, end=end)
omni_1_min = pd.DataFrame(omni_1_min)
omni_1_min = omni_1_min.set_index(keys="EPOCH", drop=True)

omni_1_hour = data_loader.load_omni_data_1hour_res(start=start, end=end)
omni_1_hour = pd.DataFrame(omni_1_hour)
omni_1_hour = omni_1_hour.set_index(keys="EPOCH", drop=True)

Loading OMNI data between: 2017-05-01 00:00:00 and 2017-06-07 23:59:59
Loaded OMNI Data for : 2017-05-01 00:00:00
Loaded OMNI Data for : 2017-06-01 00:00:00
Loading: c:\Dev\Research\REPT_Enhancements_Tool\raw_data\OMNI\_1_hour_res\omni2_h0_mrg1hr_20170101_v01.cdf
Loading: c:\Dev\Research\REPT_Enhancements_Tool\raw_data\OMNI\_1_hour_res\omni2_h0_mrg1hr_20170701_v01.cdf


In [29]:
metop1 = data_loader.load_compressed_poes_data("metop1", start=start, end=end)
metop2 = data_loader.load_compressed_poes_data("metop2", start=start, end=end)
noaa15 = data_loader.load_compressed_poes_data("noaa15", start=start, end=end)
noaa16 = data_loader.load_compressed_poes_data("noaa16", start=start, end=end)
noaa18 = data_loader.load_compressed_poes_data("noaa18", start=start, end=end)
noaa19 = data_loader.load_compressed_poes_data("noaa19", start=start, end=end)

satellites = [metop1, metop2, noaa15, noaa16, noaa18, noaa19]


Loading : POES_201705_metop1_CLEAN.npz
Loading : POES_201706_metop1_CLEAN.npz
Loading : POES_201705_metop2_CLEAN.npz
Loading : POES_201706_metop2_CLEAN.npz
Loading : POES_201705_noaa15_CLEAN.npz
Loading : POES_201706_noaa15_CLEAN.npz

Data file not found: c:\Dev\Research\REPT_Enhancements_Tool\compressed_data\POES\CLEAN\noaa16\POES_201705_noaa16_CLEAN.npz, continuing...!

Data file not found: c:\Dev\Research\REPT_Enhancements_Tool\compressed_data\POES\CLEAN\noaa16\POES_201706_noaa16_CLEAN.npz, continuing...!
Loading : POES_201705_noaa18_CLEAN.npz
Loading : POES_201706_noaa18_CLEAN.npz
Loading : POES_201705_noaa19_CLEAN.npz
Loading : POES_201706_noaa19_CLEAN.npz


In [30]:
poes_epoch = np.concatenate([sat.get_data(data_references.POES.EPOCH) for sat in satellites], axis=0)
poes_L = np.concatenate([sat.get_data(data_references.POES.L) for sat in satellites], axis = 0)
chorus = np.concatenate([sat.get_data(data_references.POES.NAIVE_CHORUS_AMPLITUDES) for sat in satellites], axis=0)

satisfies_L_cut_POES = ((l_cut - tol) < poes_L) & (poes_L < (l_cut + tol))
poes_epoch = poes_epoch[satisfies_L_cut_POES]
poes_L = poes_L[satisfies_L_cut_POES]
chorus = chorus[satisfies_L_cut_POES]

POES = pd.DataFrame({"L": poes_L, "Chorus": chorus}, index=poes_epoch).sort_index().resample("1h").mean()

In [28]:
field_model = model.TS04D

B_l = 0.311653 / (l_cut ** 3)
E_l = [0.075, 0.125, 0.250, 0.5, 1.0, 2.0]
M_e = scipy.constants.physical_constants["electron mass energy equivalent in MeV"][0]

mu_vals = [(E**2 + 2 * M_e * E) / (2 * M_e * B_l) for E in E_l]
#mu_vals = [500, 1000, 3000, 5000, 8000]
chosen_k = 0.05

In [20]:
psd_refs_A = data_loader.load_psd(satellite = "A", field_model = field_model, start = start, end = end)
psd_refs_B = data_loader.load_psd(satellite = "B", field_model = field_model, start = start, end = end)

2017-05-01 00:00:00
Loading : PSD_201705_A_TS04D.npz
2017-06-01 00:00:00
Loading : PSD_201706_A_TS04D.npz
2017-05-01 00:00:00
Loading : PSD_201705_B_TS04D.npz
2017-06-01 00:00:00
Loading : PSD_201706_B_TS04D.npz


In [21]:
for chosen_mu in mu_vals:
    
    print(f"Selecting PSD for A at mu = {chosen_mu}, k = {chosen_k}")
    
    selected_psd_refs_A = psd_tool.select_mu_and_k_from_psd(refs=psd_refs_A, chosen_mu=chosen_mu, chosen_k=chosen_k)
    
    print(f"Selecting PSD for B at mu = {chosen_mu}, k = {chosen_k}")

    selected_psd_refs_B = psd_tool.select_mu_and_k_from_psd(refs=psd_refs_B, chosen_mu=chosen_mu, chosen_k=chosen_k)

    data_cache.cache_psd_at_selected_mu_and_k(selected_psd_refs_A, satellite = "A", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    data_cache.cache_psd_at_selected_mu_and_k(selected_psd_refs_B, satellite = "B", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)


Selecting PSD for A at mu = 32.289086421340485, k = 0.05
(18240, 35, 102) (18240,) (18240,) (18240, 102) (18240, 35) (18240, 35) (18240, 35) (18240,)
Time taken for loop: 6.209099292755127
Selecting PSD for B at mu = 32.289086421340485, k = 0.05
(18240, 35, 102) (18240,) (18240,) (18240, 102) (18240, 35) (18240, 35) (18240, 35) (18240,)
Time taken for loop: 6.050835371017456
Selecting PSD for A at mu = 56.26798118482581, k = 0.05
(18240, 35, 102) (18240,) (18240,) (18240, 102) (18240, 35) (18240, 35) (18240, 35) (18240,)
Time taken for loop: 5.870164394378662
Selecting PSD for B at mu = 56.26798118482581, k = 0.05
(18240, 35, 102) (18240,) (18240,) (18240, 102) (18240, 35) (18240, 35) (18240, 35) (18240,)
Time taken for loop: 5.746605396270752
Selecting PSD for A at mu = 124.80014811594326, k = 0.05
(18240, 35, 102) (18240,) (18240,) (18240, 102) (18240, 35) (18240, 35) (18240, 35) (18240,)
Time taken for loop: 5.797901153564453
Selecting PSD for B at mu = 124.80014811594326, k = 0.05


In [22]:

slow_moving = omni_1_min["AE"].rolling("3h", center=False).mean()
#f = (OMNI["AE"] - slow_moving)
f = omni_1_min["AE"].diff()
threshold = 50
shifted_f1 = (f-threshold)
shifted_f2 = (f+threshold)

zero_crossings1 = ((shifted_f1.shift() >= 0) & (shifted_f1 < 0)) | ((shifted_f1.shift() < 0) & (shifted_f1 >= 0))
zero_crossings2 = ((shifted_f2.shift() >= 0) & (shifted_f2 < 0)) | ((shifted_f2.shift() < 0) & (shifted_f2 >= 0))
zero_crossings = zero_crossings1 | zero_crossings2
num_crossings = zero_crossings.rolling("3h", center=False).sum().rolling("3h", center=True).mean()

fig, axs = plt.subplots(6, 1, sharex=True)

num_energies = fesa.shape[1]
energy_colors = plt.cm.get_cmap("plasma", num_energies)

for E in range(num_energies):
    
    axs[0].semilogy(fesa.index, fesa[E], label=f"{energies[E]} MeV", color=energy_colors(E), linewidth=2.5)
    
axs[0].set_ylabel("Flux ($cm^{-2}s^{-1}sr^{-1}MeV^{-1}$)")
axs[0].legend(loc="upper right")
axs[0].yaxis.grid(color='gray', linestyle='dashed')
axs[0].xaxis.grid(color='gray', linestyle='dashed')
axs[0].set_xlim(start, end)
axs[0].set_title(f"L = {l_cut}, RBSP-COMBINED")

num_mu = len(mu_vals)
mu_colors = plt.cm.get_cmap("plasma", num_mu)

for k, chosen_mu in enumerate(mu_vals):
    
    selected_psd_refs_A = data_cache.load_psd_cache(satellite = "A", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    selected_psd_refs_B = data_cache.load_psd_cache(satellite = "B", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    
    epoch_A = selected_psd_refs_A["EPOCH"]
    Lstar_A = selected_psd_refs_A["L_STAR"] 
    L_A = selected_psd_refs_A["L"]
    PSD_A = selected_psd_refs_A["PSD"]
    
    epoch_B = selected_psd_refs_B["EPOCH"]
    Lstar_B = selected_psd_refs_B["L_STAR"]
    L_B = selected_psd_refs_B["L"]
    PSD_B = selected_psd_refs_B["PSD"]
    
    satisfies_L_cut_A = ((l_cut - tol) < Lstar_A) & (Lstar_A < (l_cut + tol))
    satisfies_L_cut_B = ((l_cut - tol) < Lstar_B) & (Lstar_B < (l_cut + tol))

    #satisfies_L_cut_A = ((l_cut - tol) < L_A) & (L_A < (l_cut + tol))
    #satisfies_L_cut_B = ((l_cut - tol) < L_B) & (L_B < (l_cut + tol))
    
    epoch_A_cut = epoch_A[satisfies_L_cut_A]
    PSD_A_cut = PSD_A[satisfies_L_cut_A]
    
    epoch_B_cut = epoch_B[satisfies_L_cut_B]
    PSD_B_cut = PSD_B[satisfies_L_cut_B]
        
    PSD_A_DF = pd.Series(PSD_A_cut, index=epoch_A_cut)
    PSD_A_DF = PSD_A_DF[np.isfinite(PSD_A_DF)]

    PSD_B_DF = pd.Series(PSD_B_cut, index=epoch_B_cut)
    PSD_B_DF = PSD_B_DF[np.isfinite(PSD_B_DF)]

    PSD = pd.concat([PSD_A_DF, PSD_B_DF], sort=False).sort_index()
    PSD = PSD.resample("5h").mean()
    PSD = PSD[np.isfinite(PSD)]
    
    axs[1].semilogy(PSD.index, PSD, linewidth=2, color = mu_colors(k), label=f"{int(chosen_mu)} (MeV / G)")
    
axs[1].yaxis.grid(color='gray', linestyle='dashed')
axs[1].xaxis.grid(color='gray', linestyle='dashed')
axs[1].set_title(rf"RBSP-COMBINED, K = {chosen_k} ($R_E$$G^{{1/2}}$), {field_model.name}")
axs[1].set_ylabel("PSD (c/Mev/G)")
axs[1].legend(loc = "upper right")

axs[2].yaxis.grid(color='gray', linestyle='dashed')
axs[2].xaxis.grid(color='gray', linestyle='dashed')
axs[2].plot(num_crossings.index, num_crossings, color="black", label="Crossings")
axs[2].set_ylabel("# Crossing Threshold")
axs[2].set_ylim(bottom=0)
#axs[1].hlines(y=100, xmin=start, xmax=end, color="green", label="Enhancement Trigger")

axs[3].plot(POES.index, POES["Chorus"], color="black")
axs[3].set_ylabel("Naive Chorus (pT)")


axs[4].yaxis.grid(color='gray', linestyle='dashed')
axs[4].xaxis.grid(color='gray', linestyle='dashed')
axs[4].plot(omni_1_min.index, omni_1_min["AE"], color="black", label="AE")
axs[4].plot(omni_1_min.index, slow_moving, color="blue", linewidth=2, label="Slow Average")
axs[4].set_ylabel("AE (nT)")
axs[4].set_ylim(bottom=0)
axs[4].legend(loc = "upper right")

axs[5].yaxis.grid(color='gray', linestyle='dashed')
axs[5].xaxis.grid(color='gray', linestyle='dashed')
axs[5].plot(omni_1_hour.index, omni_1_hour["DST"], color="black")
axs[5].set_ylabel("DST")
axs[5].set_xlabel("Time")


#axs[4].plot(omni.index, f, color="black", label="$|{\delta}AE|$")
#axs[4].set_ylabel("$|{\delta}AE|$ (nT)")
#axs[4].yaxis.grid(color='gray', linestyle='dashed')
#axs[4].xaxis.grid(color='gray', linestyle='dashed')
#axs[4].hlines(y=threshold, xmin=start, xmax=end, color="green", label="Threshold")
#axs[4].hlines(y=-1 * threshold, xmin=start, xmax=end, color="green", label="Threshold")

#axs[4].set_ylim(bottom=0)
#axs[4].legend()

#cbar = plt.colorbar(image_cmap, ax=axs[4])

#cbar.set_label("Bw (pT)\n", loc="center", rotation=270)

fig.set_size_inches(12, 8)

plt.show()

C:\Users\brofe\AppData\Local\Temp\ipykernel_32316\674101251.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  energy_colors = plt.cm.get_cmap("plasma", num_energies)


In [31]:

f = omni_1_min["Bz"]
threshold = 1.0

zero_crossings = (((f.shift() >= 0) & (f < 0)) | ((f.shift() < 0) & (f >= 0))) & (f.abs() > threshold) & (f.shift().abs() > threshold)
num_crossings = zero_crossings.rolling("24h", center=True).sum().resample("5h").mean()

fig, axs = plt.subplots(6, 1, sharex="all")

num_energies = fesa.shape[1]
energy_colors = plt.cm.get_cmap("plasma", num_energies)

for E in range(num_energies):
    
    axs[0].semilogy(fesa.index, fesa[E], label=f"{energies[E]} MeV", color=energy_colors(E), linewidth=2.5)


axs[0].set_ylabel("Flux ($cm^{-2}s^{-1}sr^{-1}MeV^{-1}$)")
axs[0].legend(loc="upper right")
axs[0].yaxis.grid(color='gray', linestyle='dashed')
axs[0].xaxis.grid(color='gray', linestyle='dashed')
axs[0].set_xlim(start, end)
axs[0].set_title(f"L = {l_cut}, RBSP-COMBINED")

num_mu = len(mu_vals)
mu_colors = plt.cm.get_cmap("plasma", num_mu)

for k, chosen_mu in enumerate(mu_vals):
    
    selected_psd_refs_A = data_cache.load_psd_cache(satellite = "A", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    selected_psd_refs_B = data_cache.load_psd_cache(satellite = "B", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    
    epoch_A = selected_psd_refs_A["EPOCH"]
    Lstar_A = selected_psd_refs_A["L_STAR"] 
    L_A = selected_psd_refs_A["L"]
    PSD_A = selected_psd_refs_A["PSD"]
    
    epoch_B = selected_psd_refs_B["EPOCH"]
    Lstar_B = selected_psd_refs_B["L_STAR"]
    L_B = selected_psd_refs_B["L"]
    PSD_B = selected_psd_refs_B["PSD"]
    
    satisfies_L_cut_A = ((l_cut - tol) < Lstar_A) & (Lstar_A < (l_cut + tol))
    satisfies_L_cut_B = ((l_cut - tol) < Lstar_B) & (Lstar_B < (l_cut + tol))
    
    epoch_A_cut = epoch_A[satisfies_L_cut_A]
    PSD_A_cut = PSD_A[satisfies_L_cut_A]
    
    epoch_B_cut = epoch_B[satisfies_L_cut_B]
    PSD_B_cut = PSD_B[satisfies_L_cut_B]
        
    PSD_A_DF = pd.Series(PSD_A_cut, index=epoch_A_cut)
    PSD_A_DF = PSD_A_DF[np.isfinite(PSD_A_DF)]

    PSD_B_DF = pd.Series(PSD_B_cut, index=epoch_B_cut)
    PSD_B_DF = PSD_B_DF[np.isfinite(PSD_B_DF)]

    PSD = pd.concat([PSD_A_DF, PSD_B_DF], sort=False).sort_index()
    PSD = PSD.resample("5h").mean()
    PSD = PSD[np.isfinite(PSD)]
    
    axs[1].semilogy(PSD.index, PSD, color = mu_colors(k), label=f"{int(chosen_mu)} (MeV / G)", linewidth=2.5)
    
    
axs[1].set_title(rf"RBSP-COMBINED, K = {chosen_k} ($R_E$$G^{{1/2}}$), {field_model.name}")
axs[1].set_ylabel("PSD (c/Mev/G)")
axs[1].legend(loc="upper right")

axs[2].yaxis.grid(color='gray', linestyle='dashed')
axs[2].xaxis.grid(color='gray', linestyle='dashed')
axs[2].plot(num_crossings.index, num_crossings, color="black", label="Crossings")
axs[2].set_ylabel("# Crossing Threshold")
#axs[2].set_ylim(bottom=0)

axs[3].plot(POES.index, POES["Chorus"], color="black")
axs[3].set_ylabel("Naive Chorus (pT)")
axs[3].set_xlabel("Time")

'''axs[4].plot(omni.index, f, color="black")

axs[4].yaxis.grid(color='gray', linestyle='dashed')
axs[4].xaxis.grid(color='gray', linestyle='dashed')
axs[4].set_ylabel("${\delta}B_{z}$ (nT)")
axs[4].hlines(y=0, xmin=start, xmax=end, color="black")
axs[4].hlines(y=threshold, xmin=start, xmax=end, color="green", label="Threshold")
axs[4].hlines(y=-1 * threshold, xmin=start, xmax=end, color="green", label="Threshold")'''

axs[4].plot(omni_1_min.index, omni_1_min["Bz"], color="black")
#axs[4].plot(omni_1_min.index, omni_1_min["Bz"].rolling("15min", center=True).mean(), label="Slow Average", color="red")

axs[4].yaxis.grid(color='gray', linestyle='dashed')
axs[4].xaxis.grid(color='gray', linestyle='dashed')
axs[4].set_ylabel("$B_{z}$ (nT)")
axs[4].set_ylim(-15, 15)
axs[4].legend(loc = "upper right")

axs[5].yaxis.grid(color='gray', linestyle='dashed')
axs[5].xaxis.grid(color='gray', linestyle='dashed')
axs[5].plot(omni_1_hour.index, omni_1_hour["DST"], color="black")
axs[5].set_ylabel("DST")
axs[5].set_xlabel("Time")

plt.show()



<>:81: SyntaxWarning: invalid escape sequence '\d'
C:\Users\brofe\AppData\Local\Temp\ipykernel_32316\1611077060.py:10: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  energy_colors = plt.cm.get_cmap("plasma", num_energies)
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [ ]:
f = omni_1_min["Bz"]
threshold = 1.0

zero_crossings = (((f.shift() >= 0) & (f < 0)) | ((f.shift() < 0) & (f >= 0))) & (f.abs() > threshold) & (f.shift().abs() > threshold)
num_crossings = zero_crossings.rolling("24h", center=True).sum().resample("5h").mean()

fig, axs = plt.subplots(6, 1, sharex="all")

num_mu = len(mu_vals)
mu_colors = plt.cm.get_cmap("plasma", num_mu)

for k, chosen_mu in enumerate(mu_vals):
    
    selected_psd_refs_A = data_cache.load_psd_cache(satellite = "A", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    selected_psd_refs_B = data_cache.load_psd_cache(satellite = "B", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    
    epoch_A = selected_psd_refs_A["EPOCH"]
    Lstar_A = selected_psd_refs_A["L_STAR"] 
    L_A = selected_psd_refs_A["L"]
    PSD_A = selected_psd_refs_A["PSD"]
    
    epoch_B = selected_psd_refs_B["EPOCH"]
    Lstar_B = selected_psd_refs_B["L_STAR"]
    L_B = selected_psd_refs_B["L"]
    PSD_B = selected_psd_refs_B["PSD"]
    
    satisfies_L_cut_A = ((l_cut - tol) < Lstar_A) & (Lstar_A < (l_cut + tol))
    satisfies_L_cut_B = ((l_cut - tol) < Lstar_B) & (Lstar_B < (l_cut + tol))
    
    epoch_A_cut = epoch_A[satisfies_L_cut_A]
    PSD_A_cut = PSD_A[satisfies_L_cut_A]
    
    epoch_B_cut = epoch_B[satisfies_L_cut_B]
    PSD_B_cut = PSD_B[satisfies_L_cut_B]
        
    PSD_A_DF = pd.Series(PSD_A_cut, index=epoch_A_cut)
    PSD_A_DF = PSD_A_DF[np.isfinite(PSD_A_DF)]

    PSD_B_DF = pd.Series(PSD_B_cut, index=epoch_B_cut)
    PSD_B_DF = PSD_B_DF[np.isfinite(PSD_B_DF)]

    PSD = pd.concat([PSD_A_DF, PSD_B_DF], sort=False).sort_index()
    PSD = PSD.resample("5h").mean()
    PSD = PSD[np.isfinite(PSD)]
    
    axs[1].semilogy(PSD.index, PSD, color = mu_colors(k), label=f"{int(chosen_mu)} (MeV / G)", linewidth=2.5)
